In [37]:
import hashlib
from sage.rings.finite_rings.integer_mod import IntegerMod_gmp
from sage.rings.polynomial.polynomial_zmod_flint import Polynomial_zmod_flint

# DIFINES
PRIME = 3*2**30+1
FIELD = GF(PRIME)

In [38]:
F.<x> = PolynomialRing(GF(PRIME),'x')
field_gen = FIELD.multiplicative_generator()
gen8192 = field_gen ** ((PRIME-1)/8192)
gen1024 = field_gen ** ((PRIME-1)/1024)

In [39]:
Y_INDEX = 999
a = 2 

In [40]:
def trace_calculator(a: int, trance_len: int) -> list:
    trace = [1, a]
    for i in range(2,trance_len):
        trace.append((trace[i-1] + trace[i-2])%PRIME)
    return trace

In [41]:
def polynomial_evaluation (trace: list, generator: IntegerMod_gmp) -> Polynomial_zmod_flint:
    points =[]
    for i, y in enumerate (trace): 
        points.append((generator**i, y))
    
    R = FIELD['x']
    polynomial = R.lagrange_polynomial(points)
    print(type(polynomial))
    return polynomial

In [42]:
trance_len = 1024
trace = trace_calculator(a=a,trance_len=trance_len)
#print(trace)
poly = polynomial_evaluation(trace=trace, generator=gen1024)
# Sainaty check:
assert poly(gen1024**Y_INDEX) == trace[Y_INDEX]
Y=trace[Y_INDEX]

<class 'sage.rings.polynomial.polynomial_zmod_flint.Polynomial_zmod_flint'>


In [43]:
def compositon_polynomial(poly: Polynomial_zmod_flint, trance_len: int, Y: IntegerMod_gmp, index_y: int , gen: IntegerMod_gmp):
    n=trance_len
    p1 = (poly-1)/(x-gen**0)
    co1 = fiat_shamir_random(data=str(p1))
    p2 = (poly - Y)/(x-gen**index_y)
    co2 = fiat_shamir_random(data=str(p2))
    
    # (x-g**0)(x-g**1)...(x-g**(n-1)) = x**n-1
    # 
    constrain_3_numer = poly(gen ** 2 * x) - poly(gen * x) - poly(x)
    # constrain_3_numer should divide by all powers of gen: x=g**0, .... x= g**(n-3)
    constrain_3_denom = (x**n-1) / ( (x-gen**(n-1)) * (x-gen**(n-2)) )
    # p3 = (poly(gen ** 2 * x) - poly(gen * x) - poly(x))*(x-gen**(n-1)) \
    #     * (x-gen**(n-2))*(x-gen**(n-3))/(x**n-1)
    p3 = constrain_3_numer / constrain_3_denom
    co3 = fiat_shamir_random(data=str(p3))
    return p1*co1 + p2+co2 + p3*co3

In [44]:
def merkle(points: list):
    if len(points) <= 1:
        return points[0]
    squeezed_points = []
    for i in range(len(points)//2):
        temp_string = ''.join(str(points[i*2]))
        temp_string = temp_string.join(str(points[i*2+1]))
        squeezed_points.append(sha3(temp_string))
    if len(points)%2 == 1:
        squeezed_points.append(sha3(''.join(str(points[-1]))))
    return merkle(squeezed_points)

In [45]:
def sha3(string: str):
    byte=string.encode('ascii')
    m = hashlib.sha3_256()
    m.update(byte)
    
    return m.hexdigest()

In [46]:
def fiat_shamir_random(data: str)-> int:
    rand = int.from_bytes(data.encode('ascii'), "big") 
    return rand%PRIME

In [47]:
CP = compositon_polynomial(poly=poly, trance_len=trance_len, Y=Y, index_y=Y_INDEX , gen=gen1024)

In [48]:
points = trace_calculator(2, 15)
merkle(points)

'112bda191ff7087fc2d7fda84d713e02eb8cae32817df6a2e2a41224bf81616c'

In [49]:
#CP

In [50]:
def reverse_bit(n, width = 10):
    n_binary = '{:0{width}b}'.format(n, width=width)
    return int(n_binary[::-1], 2)

In [61]:
def low_degree_extension(poly: Polynomial_zmod_flint, trance_len: int, group_gen: IntegerMod_gmp, field_gen: IntegerMod_gmp):
    coset_set = [field_gen*(group_gen**i) for i in range (trance_len)]
    new_coset_set = [coset_set[reverse_bit(i, log(trance_len,2))] for i in range (trance_len)]
    return [(c, poly(c)) for c in new_coset_set]

In [62]:
low_degree_extension(poly=CP, trance_len=16, group_gen=gen8192, field_gen=field_gen)

[(5, 240136124),
 (2833855974, 3137794864),
 (2623023147, 832770097),
 (2540449978, 178471777),
 (1056415280, 2443620285),
 (1800233432, 3212932430),
 (380960369, 1435039985),
 (671626609, 1479639957),
 (2229935889, 2287296401),
 (2457247830, 2896588392),
 (19433037, 3031081945),
 (3171820980, 546310648),
 (2439017906, 1350764941),
 (13901640, 2967064874),
 (820071082, 450352649),
 (565703797, 1281215413)]

In [63]:
def fri(poly: Polynomial_zmod_flint, domain: list , degree = 1024) -> dict:
    
    proof = {} #{stage: [merkle root, [pathes], random number for naxt-stage]}
    for stage in range (log(degree, 2)):
        
        pathes = []
        points, merkel_root = commit(poly=poly, domain=domain)
        #first stage: evaluation above the whole domain and creation of merkle tree

        evaluate_points_and_path()
        #second stage: takes n number randoms, and claculate n/2 time P(x_i) 1<i<n/2 + merkle path for them

        rand = fiat_shamir_random(merkel_root)
        proof[stage]= [merkel_root ,pathes, rand]
        poly, domain = fri_next_layer(poly=poly, domain=domain, rand = rand)
        #third stage: calculates FRI next Layer
    
    return proof

In [64]:
def commit(poly: Polynomial_zmod_flint, domain: list):
    #first stage: evaluation above the whole domain and creation of merkle tree
    points=[(d, poly(d)) for d in domain]
    return (points, merkle(points=points))

In [65]:
def fri_next_layer(poly: Polynomial_zmod_flint, domain: list , rand: int):
    #calculate the polynomial and the domain of the next stage
    even = 0*x
    odd = 0*x
    for degree,coef in poly.dict().items():
        if degree%2==0:
            even = even + coef*x**(degree//2)
        else:
            odd = odd + coef*x**(degree//2)
    next_layer = even + rand*odd
    new_domain = []
    for i in range(0,len(domain),2):
        assert domain[i]**2 == domain[i+1]**2
        new_domain.append(domain[i]**2)
    return next_layer, new_domain

In [66]:
def evaluate_points_and_path():
    pass

In [67]:
trace = trace_calculator(2,15)
poly = polynomial_evaluation(trace, field_gen ** ((PRIME-1)/16))
#poly = compositon_polynomial(poly, 15, field_gen ** ((PRIME-1)/16), 10, field_gen)
#f = 3*x**1 #2895570615*x**12 + x**7
#poly = poly.sub(f)
#f.coefficients(0)
#poly.dict()
print(f"{poly} \n \n" )
poly, domain = fri_next_layer(poly,[0,0,0,0], 1)
print (f"{poly} \n \n {254490165 + 2542374603} \n \n {domain}" )

trace = [0]*16
proof = fri(poly,trace,16)
proof

<class 'sage.rings.polynomial.polynomial_zmod_flint.Polynomial_zmod_flint'>
1949650749*x^14 + 254490165*x^13 + 2542374603*x^12 + 1167923054*x^11 + 2345897642*x^10 + 2315637714*x^9 + 2941037616*x^8 + 2682956350*x^7 + 56638226*x^6 + 854385164*x^5 + 1484157149*x^4 + 54628453*x^3 + 2895570615*x^2 + 2333655492*x + 1890800793 
 

1949650749*x^7 + 2796864768*x^6 + 292595223*x^5 + 2035449857*x^4 + 2739594576*x^3 + 2338542313*x^2 + 2950199068*x + 1003230812 
 
 2796864768 
 
 [0, 0]


{0: ['c87e08942e4e9cbd3fab82cfa4226926a81329539ce8625da2de602bffecef6b',
  [],
  1369968218],
 1: ['f358532b1d5611ca6e474d6569c808bf01b4f8d3210de943197e3b33acb0cda3',
  [],
  2733178454],
 2: ['81cb2040b6b34230ef32cc06c6bc7f73d629c932b241187e787ed558f4dda829',
  [],
  2355736069],
 3: ['ad8a8fd47b6f1a3da797f1fd55d9f85973bd47719234e83b9571722e0b50f4dc',
  [],
  3033393731]}

In [68]:
trace = []
domain = []
for i in range (1024):
    trace.append(gen1024**i)
for i in range (1024):
    domain.append(trace[reverse_bit(i, 10)])
domain[2]**2

3221225472